In [ ]:
# 1. 安装系统依赖
!sudo apt update && sudo apt install ffmpeg -y

# 2. 升级 pip 并安装最新版核心库
!pip install --upgrade pip
!pip install -U demucs stable-ts torch tqdm torchcodec

In [ ]:
# @title 🛠️ 配置中心 (Configuration & Setup)
import os
import torch
from google.colab import drive

# 1. 挂载 Google Drive
if not os.path.exists("/content/drive"):
    print("正在挂载 Google Drive...")
    drive.mount('/content/drive')

# 2. 文件夹路径设置
# @markdown ### 📂 路径设置
SOURCE_FOLDER = "/content/drive/MyDrive/audios"  # @param {type:"string"}
OUTPUT_FOLDER = "/content/drive/MyDrive/lyrics" # @param {type:"string"}

# 3. 模型识别参数
# @markdown ---
# @markdown ### 🤖 模型与识别设置
MODEL_SIZE = "large-v3" # @param ["medium", "large-v3"]
LANGUAGE = "auto" # @param ["auto", "zh", "ja", "en", "ko"]
# 自动跳过文件名包含以下关键词的歌曲
SKIP_KEYWORDS = ["伴奏", "Instrumental", "Off Vocal", "Inst", "カラオケ"] # @param {type:"raw"}

# 4. 歌词切分与精修策略
# @markdown ---
# @markdown ### ✂️ 歌词切分与精修参数 (Post-Processing)
MAX_CHARS_PER_LINE = 24 # @param {type:"integer"}
LIMIT_DUR = 3.0 # @param {type:"number"}
MIN_GAP = 0.8 # @param {type:"number"}
INNER_WORD_OFFSET_MS = 150 # @param {type:"integer"}
SYNC_THRESHOLD = 0.8 # @param {type:"number"}

# 5. 运行参数
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print(f"✅ 配置完成。处理结果将存入: {OUTPUT_FOLDER}")

In [ ]:
# @title 🚀 开启原子化自动化处理 (V11 Robust Format Detection)
import re
import gc
import json
import torch
import shutil
import subprocess
import traceback
import torchaudio
import torchaudio.transforms as T
from pathlib import Path
from datetime import datetime
from tqdm.auto import tqdm
import stable_whisper

# ==========================================
# 1. 核心工具函数
# ==========================================
def sanitize_filename(name):
    return re.sub(r'[\\/*?:"<>|]', "", name).replace(" ", "_")

def strip_punc(text):
    """移除歌词中的标点符号 - 兼容 Python 3.12"""
    punc_list = [
        '，', '。', '！', '？', '、', '；', '：', '“', '”', '‘', '’',
        '（', '）', '【', '】', '《', '》', ',', r'\.', r'\?', '!',
        r'\(', r'\)', r'\[', r'\]', '"', "'"
    ]
    pattern = "|".join(punc_list)
    return re.sub(pattern, "", text).strip()

def get_unique_output_path(original_path, output_dir):
    p = Path(original_path)
    safe_name = sanitize_filename(p.stem)
    parent_name = sanitize_filename(p.parent.name)
    return os.path.join(output_dir, f"{safe_name}_{parent_name}.json")

def get_real_audio_format(file_path):
    """使用 ffprobe 探测文件的真实封装格式"""
    try:
        cmd = [
            "ffprobe", "-v", "error", "-show_entries", "format=format_name",
            "-of", "default=noprint_wrappers=1:nokey=1", file_path
        ]
        result = subprocess.run(cmd, capture_output=True, text=True, check=True)
        fmt = result.stdout.strip().lower()
        # 处理常见的复合格式名，如 'mov,mp4,m4a,3gp,3g2,mj2'
        if 'm4a' in fmt: return '.m4a'
        if 'mp3' in fmt: return '.mp3'
        if 'wav' in fmt: return '.wav'
        if 'flac' in fmt: return '.flac'
        return '.bin' # 未知格式
    except Exception:
        return None

# ==========================================
# 2. 初始化常驻模型
# ==========================================
print("📦 正在预载识别引擎 (Whisper & VAD)...")
vad_model, vad_utils = torch.hub.load(repo_or_dir='snakers4/silero-vad', model='silero_vad', force_reload=False, trust_repo=True)
(get_speech_timestamps, _, _, _, _) = vad_utils
vad_model = vad_model.to(DEVICE)
whisper_model = stable_whisper.load_model(MODEL_SIZE, device=DEVICE)
print("✅ 引擎就绪。")

# ==========================================
# 3. 定义单文件原子处理逻辑
# ==========================================
def process_single_file(file_path, output_json_path):
    job_dir = "/content/work"
    if os.path.exists(job_dir): shutil.rmtree(job_dir)
    os.makedirs(job_dir)

    try:
        # --- A. 真实格式探测 ---
        real_ext = get_real_audio_format(file_path)
        if not real_ext:
            raise ValueError(f"无法解析音频内容，文件可能损坏: {file_path}")

        # 定义本地路径
        local_input = os.path.join(job_dir, f"temp_in{real_ext}")
        local_wav = os.path.join(job_dir, "temp_in.wav")
        vocal_out_dir = os.path.join(job_dir, "out")

        # --- B. 物理化中转与转码 ---
        shutil.copy2(file_path, local_input)

        print(f"\n🎵 格式确认({real_ext}): {Path(file_path).name}")

        # 转码为标准 PCM WAV
        ffmpeg_cmd = [
            "ffmpeg", "-y", "-i", local_input,
            "-vn", "-acodec", "pcm_s16le", "-ar", "44100", "-ac", "2",
            local_wav
        ]
        res = subprocess.run(ffmpeg_cmd, capture_output=True, text=True)
        if res.returncode != 0:
            raise RuntimeError(f"FFmpeg 转码失败: {res.stderr}")

        # --- C. 人声分离 (HTDemucs) ---
        print(f"🎸 提取纯净人声...")
        subprocess.run([
            "demucs", "-n", "htdemucs", "--two-stems", "vocals",
            "--device", "cuda", "-o", vocal_out_dir, local_wav
        ], check=True, capture_output=True)

        # 寻找分离后的人声文件
        vocal_wav_path = None
        for root, _, files in os.walk(vocal_out_dir):
            if "vocals.wav" in files:
                vocal_wav_path = os.path.join(root, "vocals.wav")
                break

        if not vocal_wav_path: raise FileNotFoundError("Demucs 未能生成人声轨道")

        # --- D. 加载音频并预处理 ---
        v_tensor, sr = torchaudio.load(vocal_wav_path)
        resampler = T.Resample(sr, 16000)
        v_16k = resampler(v_tensor).mean(0).to(DEVICE)
        v_np = v_16k.cpu().numpy()

        # --- E. 识别与语义重组 ---
        def run_transcribe(uv):
            args = {"vad": uv, "regroup": False, "language": None if LANGUAGE == "auto" else LANGUAGE}
            fn = str(file_path).lower()
            if LANGUAGE == "zh" or any(z in fn for z in ["zh", "cn", "chinese", "中文"]):
                args["initial_prompt"] = "歌词转写，无标点。"
            elif LANGUAGE == "ja" or any(j in fn for j in ["ja", "jp", "japanese", "日"]):
                args["initial_prompt"] = "歌詞転写、句読点なし。"
            return whisper_model.transcribe(v_np, **args)

        try: result = run_transcribe(True)
        except: result = run_transcribe(False)

        # 语义重组逻辑
        algo_str = f'cm_sp={MIN_GAP}'
        try: result.regroup(algorithms=algo_str)
        except:
            try: result.regroup(regroup_strategy=algo_str)
            except: result.regroup()

        result.split_by_length(max_chars=MAX_CHARS_PER_LINE)
        result.split_by_duration(max_dur=3.0)
        result.merge_by_gap(0.1, max_words=3)

        # --- F. VAD 时间轴对齐 ---
        raw_dict = result.to_dict()
        speech_ts = get_speech_timestamps(v_16k, vad_model, threshold=0.3)
        vad_segs = [{"start": st['start'] / 16000, "end": st['end'] / 16000} for st in speech_ts]

        refined = []
        off_s = INNER_WORD_OFFSET_MS / 1000.0
        for i, seg in enumerate(raw_dict.get('segments', [])):
            txt = strip_punc(seg.get('text', ''))
            if not txt: continue
            s, e = seg['start'], seg['end']
            for v in vad_segs:
                if abs(v['start'] - s) <= SYNC_THRESHOLD: s = v['start']
                if v['start'] <= e <= v['end']: e = v['end']

            words = []
            for idx, w in enumerate(seg.get('words', [])):
                wt = strip_punc(w['word'])
                if not wt: continue
                ws = w['start'] + (off_s if 0 < idx < len(seg['words'])-1 else 0)
                we = w['end'] + (off_s if 0 < idx < len(seg['words'])-1 else 0)
                words.append({
                    "word": wt, "start": round(max(s, min(ws, e)), 3), "end": round(max(s, min(we, e)), 3)
                })
            refined.append({"text": txt, "start": round(s, 3), "end": round(e, 3), "words": words})

        # --- G. 保存结果 ---
        with open(output_json_path, 'w', encoding='utf-8') as f:
            json.dump({
                "metadata": {"file": Path(file_path).name, "duration": round(len(v_np)/16000, 2), "language": raw_dict.get("language")},
                "segments": refined
            }, f, indent=2, ensure_ascii=False)
        return True

    finally:
        if os.path.exists(job_dir): shutil.rmtree(job_dir)

# ==========================================
# 4. 执行循环
# ==========================================
os.makedirs(OUTPUT_FOLDER, exist_ok=True)
error_log = os.path.join(OUTPUT_FOLDER, "error_log.txt")

# 扫描逻辑：初始过滤不依赖后缀，处理时才校验
print(f"🔍 正在扫描文件夹...")
tasks = []
for r, d, fs in os.walk(SOURCE_FOLDER):
    for f in fs:
        if not any(k.lower() in f.lower() for k in SKIP_KEYWORDS):
            # 允许尝试常见的音频后缀，具体的在处理时再探测头信息
            if f.lower().endswith(('.m4a', '.mp3', '.wav', '.flac', '.ogg', '.aac')):
                tasks.append(os.path.join(r, f))

print(f"🎵 待处理清单: {len(tasks)} 首歌曲。")

pbar = tqdm(tasks)
for f_path in pbar:
    pbar.set_description(f"Processing: {Path(f_path).stem[:20]}")
    out_j = get_unique_output_path(f_path, OUTPUT_FOLDER)
    if os.path.exists(out_j): continue

    try: process_single_file(f_path, out_j)
    except Exception as e:
        with open(error_log, "a", encoding="utf-8") as l:
            l.write(f"[{datetime.now()}] {f_path}: {str(e)}\n---\n")
        print(f"❌ 失败: {Path(f_path).name}")
    finally:
        torch.cuda.empty_cache()
        gc.collect()

print(f"✨ 处理结束！")